![](https://www.wealthandfinance-news.com/wp-content/uploads/2019/09/stocks.jpg)

Portfolio optimization is choosing how much to invest in each item in the portfolio to get the highest possible return. In this example, I will distribute R$10.000,00 ($2084 dollars) in a portfolio containing Brazilian stocks.

In [1]:
!pip install yfinance;

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
%matplotlib inline

from pandas_datareader.data import DataReader
import yfinance as yf

import datetime

In [3]:
start = datetime.datetime(2021, 1, 1)
end = datetime.datetime.now()

In [4]:
tickers = ['BTC-USD', 'ETH-USD', 'ADA-USD', 'BNB-USD', 'SOL-USD', 'DOT-USD', 'TRX-USD', 'NEAR-USD']
dataset = pd.DataFrame()

for t in tickers:
    dataset[t] = yf.download(t, start, end)['Adj Close']

In [5]:
dataset.head()

In [6]:
dataset.tail()

In [7]:
dataset.to_csv('portfolio.csv')

### Random allocation

In [8]:
df = pd.read_csv('./portfolio.csv')
df.head()

⬇️⬇️⬇️Function that randomly distributes the total amount of money invested among the assets in the portfolio⬇️⬇️⬇️

In [9]:
def asset_allocation(df, total_money, seed = 0, best_weights = []):
    df = df.copy()
    
    if seed != 0:
        np.random.seed(seed)
        
    if len(best_weights) > 0:
        weights = best_weights
    else:
        weights = np.random.random(len(df.columns) - 1)
        weights = weights / weights.sum()
        
    columns = df.columns[1:]
    for i in columns:
        df[i] = (df[i] / df[i][0])
        
    for i, stock in enumerate(df.columns[1:]):
        df[stock] = df[stock] * weights[i] * total_money
        
    df['sum value'] = df.sum(axis = 1)
    
    dates = df['Date']
    
    df.drop(labels = ['Date'], axis = 1, inplace = True)
    
    df['return rate'] = 0.0
    
    for i in range(1, len(df)):
        df['return rate'][i] = ((df['sum value'][i] / df['sum value'][i - 1]) - 1) * 100
        
    stock_weights = pd.DataFrame(data = {'Stocks': columns, 'Weights': weights * 100})
    
    return df, dates, stock_weights, df.loc[len(df) - 1]['sum value']

In [10]:
df, dates, stock_weights, sum_value = asset_allocation(pd.read_csv('./portfolio.csv'), 10000, 10)

In [11]:
df

In [12]:
stock_weights

In [13]:
sum_value

In [14]:
figure = px.line(x = dates, y = df['return rate'], title = 'Portfolio daily return')
figure.show()

In [15]:
figure = px.line(title = 'Equity evolution')
for i in df.drop(columns = ['sum value', 'return rate']).columns:
    figure.add_scatter(x = dates, y = df[i], name = i)
figure.show()

In [16]:
figure = px.line(x = dates, y = df['sum value'], title = 'Portfolio evolution')
figure.show()

⬆️⬆️⬆️This would be the return on the portfolio in 5 years if you had invested R$10,000.00⬆️⬆️⬆️

### Cumulative return over the entire period

In [17]:
df.loc[len(df) -1]['sum value'] / df.loc[0]['sum value'] - 1

### Standard deviation

In [18]:
df['return rate'].std()

### Sharp Ratio

Sharpe Ratio calculates a portfolio's return relative to risk. It is the expected return minus the risk-free rate divided by the standard deviation. Above 1 is acceptable; above 2 is good and above 3 is excellent! That is, the higher the index, the greater the return.

In [19]:
(df['return rate'].mean() / df['return rate'].std()) * np.sqrt(350)

### Markowitz Optimization

Markowitz has proven the existence of an efficient set of portfolios that optimize investors' returns for the amount of risk they are willing to accept, and how different securities in a portfolio interact with each other. By combining low-correlated bonds, investors can optimize their returns without taking on additional risk.

In [20]:
port = pd.read_csv('./portfolio.csv')
port.head()

In [21]:
import sys

In [22]:
def portfolio_allocation(port, total_money, free_risk, repetitions):
  port = port.copy()
  port_original = port.copy()

  expected_return_list = []
  expected_volatility_list = []
  sharpe_ratio_list = []

  best_sharpe_ratio = 1 - sys.maxsize
  best_weights = np.empty(len(port.columns) - 1)
  best_volatility = 0
  best_return = 0
  
  for _ in range(repetitions):
    weights = np.random.random(len(port.columns) - 1)
    weights = weights / weights.sum()

    for i in port.columns[1:]:
      port[i] = port[i] / port[i][0]

    for i, stock in enumerate(port.columns[1:]):
      port[stock] = port[stock] * weights[i] * total_money

    port.drop(labels = ['Date'], axis = 1, inplace=True)

    portfolio_return = np.log(port / port.shift(1))
    covariance_matrix = portfolio_return.cov()

    port['sum value'] = port.sum(axis = 1)
    port['return rate'] = 0.0

    for i in range(1, len(port)):
      port['return rate'][i] = np.log(port['sum value'][i] / port['sum value'][i - 1])

    
    expected_return = np.sum(port['return rate'].mean() * weights) * 350
    expected_volatility = np.sqrt(np.dot(weights, np.dot(covariance_matrix * 350, weights)))
    sharpe_ratio = (expected_return - free_risk) / expected_volatility

    if sharpe_ratio > best_sharpe_ratio:
      best_sharpe_ratio = sharpe_ratio
      best_weights = weights
      best_volatility = expected_volatility
      best_return = expected_return

    expected_return_list.append(expected_return)
    expected_volatility_list.append(expected_volatility)
    sharpe_ratio_list.append(sharpe_ratio)
    
    port = port_original.copy()

  return best_sharpe_ratio, best_weights, expected_return_list, expected_volatility_list, sharpe_ratio_list, best_volatility, best_return

In [23]:
# Selic rate: base interest rate used by the central bank of brazil to control inflation. 2017 to 2022.
historic_selic_rate = np.array([12.25, 6.5, 5.0, 2.0, 9.25, 13.25])
historic_selic_rate.mean() / 100

In [24]:
sharpe_ratio, best_weights, ls_return, ls_volatility, ls_sharpe_ratio, best_volatility, best_return = portfolio_allocation(pd.read_csv('./portfolio.csv'), 10000, historic_selic_rate.mean() / 100, 1000)

In [25]:
sharpe_ratio, best_weights

In [26]:
_, _, stock_weights, sum_value = asset_allocation(pd.read_csv('./portfolio.csv'), 10000, best_weights = best_weights)

In [27]:
stock_weights, sum_value

In [28]:
best_return, best_volatility

In [29]:
plt.figure(figsize=(10,8))
plt.scatter(ls_volatility, ls_return, c = ls_sharpe_ratio)
plt.colorbar(label = 'Sharpe ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.scatter(best_volatility, best_return, c = 'red', s = 100);

The red dot indicates the best return, i.e., the Markowitz optimization yielded a 51% return on a 30% risk, with a Sharpe ratio of 1.40, considerably better than the result obtained through the previous random allocation, which had achieved a 5-year total return of 141,000, compared to a 5-year total return of 154,000 with the Markowitz allocation! It is recommended to diversify the portfolio between fixed and variable income for a better return.